# Vehicle Route Planning

## Packages

In [36]:
## Data Acess and Manipulation
import json
import numpy as np
import pandas as pd
import snowflake.connector as sf

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## for vehicle route planning
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

## Utilities
import os
import logging
from functools import partial
#from google.colab import userdata
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor as Executor

## Geospatial Work
import folium
from folium import plugins
import osmnx as ox
import networkx as nx
from shapely.geometry import shape, Point
from math import sin, cos, sqrt, atan2, radians

## Multiprocessing
from multiprocessing import Pool, cpu_count

## Utilities
### importing necessary functions from dotenv library and loading variables from .env file
import os
from dotenv import load_dotenv 
load_dotenv()

# Geopandas
import geopandas

from tqdm import tqdm

import plotly.express as px

In [37]:
# Notebook Configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [38]:
#>> Contains layout and callbacks for app 1
''' 
Notes:
This file is for creating app 1, it ontains layout and callbacks for app 1
'''
# Import packages---------------------------------------------------------------------------------------------
## Data Acess and Manipulation
import numpy as np

## Import App Packages
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
from datetime import datetime as dt
from dash.exceptions import PreventUpdate
import pandas as pd
import io
from flask import send_file

## Data Visualization
import plotly.graph_objs as go

##Local Imports
from src.utils.data_loaders import get_data_from_sf, get_geojson, load_dict
from src.utils.queries import data_query_order_line, data_query_item_weights
from src.utils.data_processing import data_preprocessing, calculate_metrics, generate_vehicle_names, data_preprocessing_map
from src.utils.map import maps, route_maps
from src.utils.vehicle_route_planning import DistanceCalculator, create_data_model, run_model
from src.utils.routes_processing import enhance_optimized_route

from app import app

## Utilities
import os
from dotenv import load_dotenv 
load_dotenv()

# Incorporate data---------------------------------------------------------------------------------------------

True

## Data


### Item Weights Master

In [39]:
# Weights Master
df_weights = pd.read_excel(
    r'assets\data\Item Weight.xlsx'
    )

print(f"Shape of the imported dataset is: {df_weights.shape}")
df_weights.head(5)

Shape of the imported dataset is: (677, 2)


,itemCode,itemWeight
0,GOSF0001,10.6
1,DBRF0006,0.1
2,AEHG0009,1.0
3,GRND0001,0.4
4,ULPL0014,1.0


### Order (Line Level)

In [40]:
data_query = '''
        WITH
        entries as
        (   select
                regexp_replace("SOURCE NO_", '[A-Z]') AS "customerCode",
                "ITEM LEDGER ENTRY NO_" as "itemLedgerEntryNo",
                "EXTERNAL DOCUMENT NO_" AS "externalDocumentNo",
                "ITEM NO_" AS "itemCode",
                "DESCRIPTION" as "itemDescription",
                "LOCATION CODE" as "locationCode",
                "O ITEM LEDGER ENTRY QUANTITY"*-1 as "invoiceQuantity",
                "O SALES AMOUNT" as "saleValue",
                "O COST AMOUNT" as "costValue",
                MIN("O DOCUMENT DATE") OVER (PARTITION BY "EXTERNAL DOCUMENT NO_") as "invoiceDate"
            from buygro.ads.adj_vle
            where "O DOCUMENT DATE" = '2024-04-03' --CURRENT_DATE()
        ),
        customer_details as
        (
            select distinct
                "ecvId",
                "erpId",
                "source",
                "name",
                replace(parse_json("details"):city, '"', '') as "city",
                replace(parse_json("details"):region, '"', '') as "region",
                replace(parse_json("details"):address, '"', '') as "address",
                parse_json("details"):location:lat as "lattitude",
                parse_json("details"):location:long as "longitude"
            from buygro.raw_ecv.bg_ecv_users
        ),
        item_details as
        (
            select
                item_code,
                description,
                min(physical_wt/physical_qty) as item_weight
            from BUYGRO.RAW.AFSYS_V_STK_DETAIL_WITH_DMG_DETAILS
            where customer = 'BUYGRO'
            and bin <> 'DESBIN'
            group by 1, 2
        )
        select ---top 2000
            b."name" as "customerName",
            b."source" as "customerSource",
            b."city" as "customerCity",
            b."region" as "customerRegion",
            b."address" as "customerAddress",
            b."lattitude" as "customerLat",
            b."longitude" as "customerLong",
            c.item_weight as "itemWeight",
            c.item_weight*"invoiceQuantity" as "itemTotalWeight",
            a.*
        from entries as a
        left join customer_details as b on a."customerCode" = b."ecvId"
        left join item_details as c on a."itemCode" = c.item_code

'''

In [41]:
conn = sf.connect(
    user=os.getenv("sf_user"),
    password=os.getenv("sf_pwd") ,
    account=os.getenv("sf_account") ,
    database='BUYGRO',
    warehouse='BUYGRO_ANALYSIS',
    schema='RAW_ECV'
)

cur = conn.cursor()
cur.execute(data_query)
df_t = pd.DataFrame(cur.fetchall(), columns=[x[0] for x in cur.description])
cur.close()
conn.close()

print(f"Shape of the imported dataset is: {df_t.shape}")

Shape of the imported dataset is: (3194, 19)


In [42]:
#df_t = get_data_from_sf(data_query_order_line)
df_t['itemTotalWeight'] = df_t['itemTotalWeight'].replace(0, 1)
df = df_t[:].copy() 
df = df[df['customerCode']!='17658'].copy() 

df.head(2)

,customerName,customerSource,customerCity,customerRegion,customerAddress,customerLat,customerLong,itemWeight,itemTotalWeight,customerCode,itemLedgerEntryNo,externalDocumentNo,itemCode,itemDescription,locationCode,invoiceQuantity,saleValue,costValue,invoiceDate
0,ZAINAB GROCERY,mobile,Ajman,Ajman,Nuaimiyah 1 Al Enjaaz Street Ajman,25.37995077,55.45546307,1.0,4.0,14807,1885725,20352326,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,4.00000000000000000000,14.48000000000000000000,-13.67000000000000000000,2024-04-03
1,HAMDA SUPERMARKET,SalescoutApp,Al Rifa,Fujairah,"Al Rifa‘ Street,Near Safeena Supermarket",25.5755402,56.2431643,1.0,5.0,20221829,1885213,20352240,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,5.00000000000000000000,18.10000000000000000000,-17.08000000000000000000,2024-04-03


In [6]:
'''# Loading Data
df_order_line = get_data_from_sf(data_query_order_line)
#df_weights = get_data_from_sf(data_query_item_weights)
js = get_geojson(os.getenv("geojson_loc_2"))

# Preprocessing the Data
df = data_preprocessing(df_order_line, df_weights, js)'''

'# Loading Data\ndf_order_line = get_data_from_sf(data_query_order_line)\n#df_weights = get_data_from_sf(data_query_item_weights)\njs = get_geojson(os.getenv("geojson_loc_2"))\n\n# Preprocessing the Data\ndf = data_preprocessing(df_order_line, df_weights, js)'

### ADS

In [43]:
# Copy
df_copy = df.copy()

# Merge weights master with orders
df = df.merge(df_weights, on='itemCode', how='left')
df['itemWeight'] = np.where(df.itemWeight_y.isna(), df.itemWeight_x, df.itemWeight_y)
df.drop(['itemWeight_x', 'itemWeight_y','itemTotalWeight'], axis=1, inplace=True)
df['invoiceQuantity'] = df['invoiceQuantity'].astype(float)
df['saleValue'] = df['saleValue'].astype(float)
df['costValue'] = df['costValue'].astype(float)
df['itemTotalWeight'] = df['itemWeight']*df['invoiceQuantity']
df.head(5)

,customerName,customerSource,customerCity,customerRegion,customerAddress,customerLat,customerLong,customerCode,itemLedgerEntryNo,externalDocumentNo,itemCode,itemDescription,locationCode,invoiceQuantity,saleValue,costValue,invoiceDate,itemWeight,itemTotalWeight
0,ZAINAB GROCERY,mobile,Ajman,Ajman,Nuaimiyah 1 Al Enjaaz Street Ajman,25.37995077,55.45546307,14807,1885725,20352326,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,4.0,14.48,-13.67,2024-04-03,1.0,4.0
1,HAMDA SUPERMARKET,SalescoutApp,Al Rifa,Fujairah,"Al Rifa‘ Street,Near Safeena Supermarket",25.5755402,56.2431643,20221829,1885213,20352240,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,5.0,18.10,-17.08,2024-04-03,1.0,5.0
2,ARDH AL SALAM GROCERY,SalescoutApp,Qasimiya,Sharjah,"26 St - Al Qasimia - Al Mahatah, Near Everyday...",25.2489204,55.3060550999999,20221695,1883367,20351951,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,3.0,10.86,-10.25,2024-04-03,1.0,3.0
3,BAQALA ALHANAN OASIS,mobile,Abu Dhabi,Abu Dhabi,Shabiya 12 6 يدالة - Mohamed Bin Zayed City - ...,24.3186035,54.5372453,6286,1883769,20352025,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,6.0,21.39,-20.50,2024-04-03,1.0,6.0
4,NEW ZUBAID SUPERMARKET,mobile,Kalba,Fujairah,"Kalba Fuajirah, Near Falcon Pack",25.07622899,56.35626177,18136,1883334,20351948,ARLA0008,"Kraft Cheddar Cheese Can, 100 gm",NIP,12.0,42.79,-41.00,2024-04-03,1.0,12.0


In [44]:
# load GeoJSON file containing sectors
with open(r'assets\data\GeoPloygon Data\GeoJSON\boundary-polygon-lvl2.geojson') as f:
    js = json.load(f)

def point_in_poly(js, latitude, longitude):
    try:
        point = Point(longitude, latitude)
    except TypeError:
        return (latitude, longitude)
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return True

df['customLocValidFlg'] = df.apply(lambda x: point_in_poly(js, x['customerLat'], x['customerLong']), axis=1)

In [45]:
# Placeholder to enrcich the data using geojson information

In [46]:
print(f"Shape of data queried: {df.shape}")

print(f"Total number of rows associated to returns: {df[df['invoiceQuantity'] < 0].shape} will be dropped\n")
df.drop(df[df['invoiceQuantity'] < 0].index, inplace=True)

print(f"Total Number of rows with null weight: {df['itemWeight'].isna().sum(axis=0)} will be dropped\n")
df[df['itemWeight'].isna()]
df.dropna(subset=['itemWeight'], inplace=True)

print(f"Total Number of rows with null customer location: {df[(df['customerLong'] == '0') & (df['customerLat'] == '0')].shape} will be dropped\n")
df.drop(df[(df['customerLong'] == '0') & (df['customerLat'] == '0')].index, inplace=True)

print(f"Total Number of rows with incorrect location {df[df['customLocValidFlg'] != True].shape} will be dropped\n")
df.drop(df[df['customLocValidFlg'] != True].index, inplace=True)

print(f"Shape of data after preprocessing: {df.shape}")

Shape of data queried: (3182, 20)
Total number of rows associated to returns: (107, 20) will be dropped

Total Number of rows with null weight: 8 will be dropped

Total Number of rows with null customer location: (11, 20) will be dropped

Total Number of rows with incorrect location (0, 20) will be dropped

Shape of data after preprocessing: (3056, 20)


In [24]:
# Loading Data
df_order_line = get_data_from_sf(data_query_order_line)
#df_weights = get_data_from_sf(data_query_item_weights)
df_order_line = df_order_line[df_order_line['customerCode']!='17658'].copy() 
df_order_line['itemTotalWeight'] = df_order_line['itemTotalWeight'].replace(0, 1)
    
js = get_geojson(os.getenv("geojson_loc_2"))

# Preprocessing the Data
df = data_preprocessing(df_order_line, df_weights, js)

## Exploration

In [47]:
# Gallega Cold Store
start = [24.931690466392208, 55.06185223067843]
order_details = df.groupby(['customerCode', 'customerName', 'customerAddress', 'customerRegion', 'customerLat', 'customerLong']
                            ).agg({
                                'externalDocumentNo': lambda x: pd.Series.nunique(x),
                                'itemTotalWeight': 'sum',
                                'itemCode': lambda x: pd.Series.nunique(x),
                                'invoiceQuantity': 'sum',
                                'saleValue': 'sum',
                                'costValue': 'sum'
                                }
                            ).reset_index().copy()
order_details.rename(columns={
                            'externalDocumentNo': 'totalOrders',
                            'itemTotalWeight': 'totalWeight',
                            'itemCode': 'totalItems'
                            }, 
                    inplace=True
                    )
order_details['invoiceQuantity'] = order_details['invoiceQuantity'].abs()
order_details['costValue'] = order_details['costValue'].abs()
order_details['grossProfit'] = order_details['saleValue'] - order_details['costValue']
order_details['grossProfitMargin'] = order_details['grossProfit'] / order_details['saleValue'] * 100
order_details['customerRegion'] = order_details['customerRegion'].str.upper()
order_details['totalWeight'] = order_details['totalWeight'].fillna(0) # Replace NaN values in the itemTotalWeight column with 0

# Adding base row
base_row = {'customerCode': 0000,'customerName': 'base', 'customerAddress': 'base', 'customerRegion': 'base', 'customerLat': start[0], 'customerLong': start[1]}
order_details = pd.concat([pd.DataFrame(base_row, index=[0]), order_details], ignore_index=True)
order_details["base"] = order_details["customerName"].apply(lambda x: 1 if x=='base' else 0)

# Replace NaN values in the itemTotalWeight column with 0
order_details['totalWeight'].fillna(0, inplace=True)
order_details.head()

C:\Users\lakshmi.n.GAGROUP\AppData\Local\Temp\ipykernel_14964\2996831854.py:33: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,customerCode,customerName,customerAddress,customerRegion,customerLat,customerLong,totalOrders,totalWeight,totalItems,invoiceQuantity,saleValue,costValue,grossProfit,grossProfitMargin,base
0,0,base,base,base,24.93169,55.061852,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1
1,10000,AL NADA GROCERY-SOLE PROPRIETORSHIP LLC,Delma Street Opp Zayed University Ewan Curtin ...,ABU DHABI,24.4648216666666,54.3827283333333,2.0,42.460000,4.0,55.0,287.71,278.71,9.00,3.128150,0
2,10028,EPEE BAQALA,57 A˜a A˜a A˜a A˜a A˜a A™a€za˜a A™a A˜a A™a A™...,ABU DHABI,24.4795116666666,54.3590733333333,2.0,264.614286,13.0,102.0,339.00,326.89,12.11,3.572271,0
3,10036,ABU MOOSA GROCERY,Dibba Fujairah Near Al Marwa Supermarket,FUJAIRAH,25.5935375,56.2727644,1.0,10.360000,4.0,27.0,148.96,142.43,6.53,4.383727,0
4,10103,Tawoon Al Madina Supermarket,Al Khan Mamzar Corniche Road Al Tawoon Sharjah,SHARJAH,25.31704624,55.36484446,1.0,44.240000,9.0,44.0,480.29,456.43,23.86,4.967832,0


In [48]:
class DistanceCalculator:
    def __init__(self, bbox=None, network_type="drive"):
        self.G = None
        self.bbox = bbox
        self.network_type = network_type

    def create_graph(self):
        if self.bbox is None:
            raise ValueError("Bounding box is not set.")
        north, south, east, west = self.bbox
        self.G = ox.graph_from_bbox(north, south, east, west, network_type=self.network_type)
        self.G = ox.add_edge_speeds(self.G)
        self.G = ox.add_edge_travel_times(self.G)
        logger.info("Graph created")

    def calculate_duration(self, a, b, units='travel_time', duration=True):
        if self.G is None:
            self.create_graph()
        if a == b:
            return 0
        try:
            route = nx.shortest_path(self.G, source=a, target=b, method='dijkstra', weight=units)
            attrs = ox.utils_graph.route_to_gdf(self.G, route)
        except nx.NetworkXNoPath:
            return np.nan
        except Exception as e:
            logger.error(f"Error {e} occurred for inputs {a}, {b}")
            return np.nan

        return attrs[units].sum() if duration else attrs['length'].sum()

    @staticmethod
    def haversine_dist( x, y):
        # Approximate radius of earth in km
        R = 6373.0

        lat1 = radians(float(x[0]))
        lon1 = radians(float(x[-1]))
        lat2 = radians(float(y[0]))
        lon2 = radians(float(y[-1]))

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c

        return distance

    @staticmethod
    def calculate_gd(args):
        x, y = args
        return DistanceCalculator.haversine_dist(x, y)

    def pairwise_distance(self, args, method='GD'):
        p1, p2 = args
        if method == 'GD':
            return self.calculate_gd((p1, p2))
        elif method == 'GRAPH':
            return self.calculate_duration(p1, p2)

    def df_to_dm(self, df, lat_col, long_col, uid, demand_ref, base_flag, method='GD'):
        if not isinstance(df, pd.DataFrame):
            raise ValueError("df must be a pandas DataFrame")

        required_cols = [lat_col, long_col, uid, demand_ref, base_flag]
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"DataFrame must contain columns: {required_cols}")

        xy_coordinates = list(map(lambda x: list(x), df[[lat_col, long_col]].values))
        xy_ids = df[uid].to_list()
        xy_demand = list(df[demand_ref].values.flatten())
        xy_demand[0] = 0

        max_workers = cpu_count()
        chunksize = 100  # Experiment with this value

        if method == 'GD':
            all_pairs = [(p1, p2) for p1 in xy_coordinates for p2 in xy_coordinates]
        elif method == 'GRAPH':
            if self.G is None:
                self.create_graph()
            xy_coordinates = list(map(lambda x: ox.distance.nearest_nodes(self.G, x[1], x[0]), xy_coordinates))
            all_pairs = [(p1, p2) for p1 in xy_coordinates for p2 in xy_coordinates]

        #with Executor(max_workers=max_workers) as executor:
        dm = list(map(partial(self.pairwise_distance, method=method), all_pairs))
        dm_matrix = np.array(dm).reshape(len(xy_coordinates), len(xy_coordinates))

        # Normalizing data for optumizer
        dm_matrix = dm_matrix*100
        dm_matrix = dm_matrix.astype(int)
        xy_demand = list(map(int, xy_demand))

        logger.info(f"Average distance between the points: {dm_matrix.mean()}")
        logger.info(f"Range distance between the points: {dm_matrix.min()} - {dm_matrix.max()}")

        return xy_ids, xy_coordinates, xy_demand, dm_matrix

In [49]:
# Usage
bbox = (22.4969475367, 26.055464179, 51.5795186705, 56.3968473651)  # Set only if using 'GRAPH' method
distance_calculator = DistanceCalculator(bbox=bbox, network_type="drive")
id, coordinates, demand, dm= distance_calculator.df_to_dm(order_details, 'customerLat', 'customerLong', 'customerCode', 'totalWeight', 'base', 'GD')

## Constraint Optimization

In [50]:
def get_num_trucks(total_weight):
    # Calculate the ideal weights based on a 70-30 mix
    weight_3ton = 0.7 * total_weight
    weight_1ton = 0.3 * total_weight

    # Calculate the number of trucks, rounding to the nearest whole number
    num_3ton_trucks = round(weight_3ton / 3000)
    num_1ton_trucks = round(weight_1ton / 1000)

    # Calculate the total weight these trucks can carry
    total_carried_weight = (num_3ton_trucks * 3000) + (num_1ton_trucks * 1000)

    # Adjusting for the remainder if total carried weight is less than total weight
    while total_carried_weight < total_weight:
        # Check which addition gets closer to the goal
        if (total_weight - total_carried_weight) >= 3000:
            num_3ton_trucks += 1
            total_carried_weight += 3000
        else:
            num_1ton_trucks += 1
            total_carried_weight += 1000

    return [3000] * num_3ton_trucks + [1000] * num_1ton_trucks + [1000]*1

truck_weights = get_num_trucks(sum(demand))
len(truck_weights) , sum(truck_weights)

(30, 56000)

In [51]:
truck_weights

[3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 3000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000]

In [52]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    vehicle_capacities = get_num_trucks(sum(demand))
    data["distance_matrix"] = dm
    data["demands"] = demand
    data["vehicle_capacities"] = vehicle_capacities
    data["num_vehicles"] = len(vehicle_capacities)
    data["depot"] = 0
    return data

In [53]:

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    routes_dict = {}
    #print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        route_seq=[]
        route_seq_dict = {}
        index = routing.Start(vehicle_id)
        # plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_distance_list = []
        route_load = 0
        route_load_list = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            # plan_output += f" {node_index} Load({route_load}) -> "
            route_seq.append(node_index)
            route_load_list.append(route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        # plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        # plan_output += f"Distance of the route: {route_distance}m\n"
        # plan_output += f"Load of the route: {route_load}\n"
        # print(plan_output)
        total_distance += route_distance
        total_load += route_load
        route_seq_dict['route_plan'] = route_seq
        route_seq_dict['cumulative_route_load'] = route_load_list
        # route_seq_dict['route_distance'] = route_distance_list
        routes_dict[vehicle_id] =  route_seq_dict
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    return routes_dict


In [54]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [55]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)
search_parameters.time_limit.FromSeconds(30)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    logger.info("Solution found!")
    output = print_solution(data, manager, routing, solution)
else:
    logger.info("No solution found!")

Total distance of all routes: 426143m
Total load of all routes: 53764


In [56]:
print(solution)

Assignment(Capacity0 (0) | Capacity1 (506) | Capacity2 (1123) | Capacity3 (2664) | Capacity4 (2835) | Capacity5 (0) | Capacity6 (499) | Capacity7 (1133) | Capacity8 (1544) | Capacity9 (1655) | Capacity10 (1989) | Capacity11 (2813) | Capacity12 (639) | Capacity13 (777) | Capacity14 (0) | Capacity15 (758) | Capacity16 (417) | Capacity17 (1634) | Capacity18 (2925) | Capacity19 (1427) | Capacity20 (688) | Capacity21 (111) | Capacity22 (1593) | Capacity23 (1724) | Capacity24 (521) | Capacity25 (0) | Capacity26 (1876) | Capacity27 (207) | Capacity28 (0) | Capacity29 (799) | Capacity30 (2628) | Capacity31 (166) | Capacity32 (1076) | Capacity33 (66) | Capacity34 (638) | Capacity35 (31) | Capacity36 (270) | Capacity37 (1387) | Capacity38 (335) | Capacity39 (2824) | Capacity40 (326) | Capacity41 (28) | Capacity42 (2206) | Capacity43 (1721) | Capacity44 (1315) | Capacity45 (732) | Capacity46 (750) | Capacity47 (974) | Capacity48 (72) | Capacity49 (0) | Capacity50 (83) | Capacity51 (161) | Capacit

In [57]:
output = print_solution(data, manager, routing, solution)

Total distance of all routes: 426143m
Total load of all routes: 53764


In [58]:
from src.utils.routes_processing import enhance_optimized_route
route_output = enhance_optimized_route(output, coordinates, id)

KeyboardInterrupt: 

## Getting routes

In [26]:
import requests
import json
import time

def reverse_coordinates(input):
  output = []
  for each in input:
    each.reverse()
    output.append(each)
  return output

def return_coords(coords,ls):
    r_ls = [coords[each] for each in ls]
    r_ls = list(map(list,r_ls))
    r_ls = reverse_coordinates(input=r_ls)
    return r_ls

def return_ids(ids,ls):
    r_ls = [ids[each] for each in ls]
    return r_ls

def call_api(coordinates):

  body = {"coordinates": coordinates}

  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': '5b3ce3597851110001cf624822b79fbe37204f6a83490d4a1566d9a5',
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)

  # print(call.status_code, call.reason)
  if call.status_code == 200:
    response = json.loads(call.text)
    geometry = response['routes'][0]['geometry']
    distance = round(response['routes'][0]['summary']['distance']/1000,1)
    duration = round(response['routes'][0]['summary']['duration']/60,1)
  else:
    print("API call failed for this list of coordinates coordinates: ", coordinates)
    print(call.text)
    geometry = None
    distance = None
    duration = None

  return geometry, distance, duration

def enhance_optimized_route(op, coords, ids):
  output = {}
  for k, v in op.items():
    output[k] = {}
    for k2, v2 in v.items():
      time.sleep(2)
      if len(return_coords(coords ,op[k]['route_plan'])) > 1:
        api_response = call_api(coordinates=return_coords(coords, op[k]['route_plan']))
        output[k]['route_geometry'] = api_response[0]
        output[k]['route_distance'] = api_response[1]
        output[k]['route_duration'] = api_response[2]
      else:
        output[k]['route_geometry'] = None
        output[k]['route_distance'] = None
        output[k]['route_duration'] = None

      output[k]['route_plan'] = op[k]['route_plan']
      output[k]['cumulative_route_load'] = op[k]['cumulative_route_load']
      output[k]['route_distance'] = 0
      output[k]['route_coords'] = return_coords(coords, op[k]['route_plan'])
      output[k]['route_ids'] = return_ids(ids, op[k]['route_plan'])

  return output

In [27]:
route_output = enhance_optimized_route(output, coordinates, id)

In [28]:
route_output_df=pd.DataFrame.from_dict(route_output, orient='index')

In [62]:
route_output_df[
    [
        'route_distance', 'route_duration', 'route_plan',
        'cumulative_route_load', 'route_coords', 'route_ids'
        ]
    ].head(2)
route_output_df.head()

,route_geometry,route_distance,route_duration,route_plan,cumulative_route_load,route_coords,route_ids
0,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,124.4,"[0, 5, 189, 82, 304, 48, 63, 262, 135, 18, 253...","[0, 21, 31, 53, 72, 90, 1310, 2916, 2925, 2934...","[[55.06185223067843, 24.931690466392208], [55....","[0, 10140, 20221674, 16064, 8059, 12845, 14225..."
1,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,366.4,"[0, 260, 291, 152, 51, 31, 305, 160, 212, 17, ...","[0, 16, 49, 161, 166, 195, 396, 396, 1634, 165...","[[55.06185223067843, 24.931690466392208], [55....","[0, 3755, 6822, 19420, 13095, 11268, 8073, 196..."
2,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,241.4,"[0, 49, 301, 333, 70, 287, 278, 263, 283, 284,...","[0, 77, 150, 212, 418, 485, 504, 1583, 1589, 1...","[[55.06185223067843, 24.931690466392208], [54....","[0, 12906, 7849, 9576, 15097, 6286, 5608, 4524..."
3,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,155.3,"[0, 141, 339, 235, 36, 176, 150, 73, 6, 1, 222...","[0, 30, 73, 270, 292, 345, 345, 499, 506, 548,...","[[55.06185223067843, 24.931690466392208], [54....","[0, 18392, 9871, 20223114, 11524, 20221179, 19..."
4,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,573.4,"[0, 14, 41, 35, 77, 67, 123, 132, 109, 101, 89...","[0, 28, 31, 61, 73, 85, 210, 348, 360, 376, 38...","[[55.06185223067843, 24.931690466392208], [55....","[0, 10428, 11779, 11419, 15589, 14713, 17719, ..."


In [63]:
route_output_df['route_coords'][1]
print(route_output_df['route_geometry'][1])

skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAqAaAsAw@qAo@oAaE_Lm@yA}AsDyAaDAKa@}@Ys@Ok@u@_ESaAW}@EaAHg@ASKg@IQMOQKQGQC_@@SDOHOLKNIREXAZDX?x@CTITmDrGiIlOeIlO_@d@y@rA{@lAs@z@e@d@_Ar@cAj@m@VeAXsANmBFiEC}@C}@KoAWuAi@u@_@cAw@sAcBk@aAqAqCIE}AyDyBqEa@w@sByDkB_Dq@_AoA}AOM_@U_GqHcAiA{E_F}CqCwCaCmDiC}EgDmKwFgVyLeHyD{OmHwMaH_GwCaIyDgJuDuGyBqHuBwDaAyD{@}Dw@sGeAuDc@ySyBeTmBcQuAoj@gFoi@_Fka@oDy\aDeGo@{Ek@mASqB[cFcA{D{@{EoAqC{@sGwBuCiAuCqA_@Q_@OsHoDgGiDuGeEkCiBkEgDmS{P}QmOcWmT}QqOi]yYwImHgLoJeZaW{WyToJqI{HuG}U{RaIaH{EcEsEuD_HgGkDiDeDoDcDqDyC{Dy@eAs^wd@{J_MeGeH{MuPwGgIkDkEkDcEoEyFkCwCyVe[eOgRgFgGaJcLsBoCmBsC_EiG_Wca@kNwTyBoD}DoGaEeHQ[gGyJcXsb@eT{\wLuRaFiI}BcDaAoAgCyCmCqCuEwDuB{AyBwA{@g@s^}RyToLkSuKcD_BgX}NoZcPeIsEeJaFuMwIiBoAkB{A_K}Ia`@o]gKqJ{MiLyDoDaTiRcUeS_O{MeF}EyNqMuMqLqRyPiIsHOOcN{LgHgG{D_DuFwDmD_CwD_Cyq@}a@mQyKoJwFcTkMwCeBaNeIiEsCqT{M_@I_@OaCwAu@m@sD}Bw@o@}@{@_@e@cA{Aw@{Ac@kAi@mBc@qB]sBc@qDWcBqBgLgA_HOsAKsAsBgNO_BKoA_AyKQ}@S}@Y{@Q_@sBqDi@o@oAgAi@_@{Au@sBs@eJqBaE}@KOGEuAa@y@[qAw@kAgA}BoCaJ}HsAiAA

In [64]:
from openrouteservice import convert
decoded = convert.decode_polyline(route_output_df['route_geometry'][3])
m = folium.Map(location=start, tiles="Cartodb Positron", zoom_start=12)
folium.GeoJson(decoded).add_to(m)
m

In [65]:
print(decoded)

{'type': 'LineString', 'coordinates': [[55.06249, 24.9313], [55.06272, 24.93161], [55.06325, 24.93231], [55.06342, 24.93248], [55.06361, 24.93262], [55.0637, 24.93262], [55.06384, 24.93257], [55.06633, 24.93104], [55.06636, 24.93108], [55.06696, 24.93185], [55.06731, 24.93227], [55.06772, 24.93266], [55.06814, 24.93299], [55.06855, 24.93327], [55.06895, 24.93351], [55.07103, 24.93448], [55.07148, 24.93471], [55.07238, 24.93518], [55.07319, 24.93563], [55.07325, 24.93564], [55.07356, 24.93581], [55.07382, 24.93594], [55.07404, 24.93602], [55.075, 24.93629], [55.07533, 24.93639], [55.07564, 24.93651], [55.07597, 24.93654], [55.07617, 24.93649], [55.07627, 24.9365], [55.07647, 24.93656], [55.07656, 24.93661], [55.07664, 24.93668], [55.0767, 24.93677], [55.07674, 24.93686], [55.07676, 24.93695], [55.07675, 24.93711], [55.07672, 24.93721], [55.07667, 24.93729], [55.0766, 24.93737], [55.07652, 24.93743], [55.07642, 24.93748], [55.07629, 24.93751], [55.07615, 24.93752], [55.07602, 24.93749], 

In [66]:
from openrouteservice import convert

def html_popup_series(row):
    try:
        html = f"""
            <h3> Customer Details </h3><br>
            <ul>
            <li>Customer Code = {row['customerCode'].values[0]}</li>
            <li>Customer Address = {row['customerAddress'].values[0]}</li>
            <li>Total Orders = {row['totalOrders'].values[0]}</li>
            <li>Total Order weight = {row['totalWeight'].values[0]}</li>
            <li>Total Items = {row['totalItems'].values[0]}</li>
            <li>Sale Value = {row['saleValue'].values[0]}</li>
            <li>Gross Profit Margin = {row['grossProfitMargin'].values[0]}</li>
            </ul>
            """
        return html
    except IndexError:
        return None

i = 1
# Locals
coordinates = route_output_df['route_coords'][i]
ids = route_output_df['route_ids'][i]
duration = route_output_df['route_duration'][i]
cumulative_route_load = route_output_df['cumulative_route_load'][i]
geometry = route_output_df['route_geometry'][i]
decoded = convert.decode_polyline(geometry)

distance_txt = "<h4> <b>Distance :&nbsp" + "<strong>"+str(cumulative_route_load)+" Km </strong>" +"</h4></b>"
duration_txt = "<h4> <b>Duration :&nbsp" + "<strong>"+str(duration)+" Mins. </strong>" +"</h4></b>"

m = folium.Map(location=start, tiles="Cartodb Positron", zoom_start=12)
folium.GeoJson(decoded).add_child(folium.Popup(distance_txt+duration_txt,max_width=300)).add_to(m)

for each in range(len(coordinates)):
    folium.Marker(
        location=list(coordinates[each][::-1]),
        popup=html_popup_series(order_details[order_details['customerCode']==str(ids[each])]),
        icon=folium.Icon(color="green"),
    ).add_to(m)

m

In [67]:
fig = px.choropleth(df_area_sales_2, 
                    geojson=js, 
                    locations="GID_2", 
                    featureidkey="properties.GID_2", 
                    color='itemTotalWeight',
                    color_continuous_scale="ylgn",  # Set color scale to green
                    range_color=(0, 1540),
                    scope='asia'
                   )

# Add points to the choropleth map
scatter_trace = px.scatter_geo(order_details, 
                               lat='customerLat', 
                               lon='customerLong', 
                               size='totalWeight',
                               color='totalWeight',
                               color_continuous_scale="Reds",  # Set color scale to red
                               opacity=.5,
                              )

scatter_trace.update_traces(marker=dict(color='Orange', size=15, line=dict(width=1, color='White')), selector=dict(mode='markers'))
fig.add_trace(scatter_trace.data[0])

# Update layout for better formatting
fig.update_layout(
    coloraxis_colorbar=dict(title='Weight Scale'),  # Adjust color bar title
    geo=dict(projection_type='mercator'  # Adjust projection type if necessary
            ),
    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, plot_bgcolor='black')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

NameError: name 'df_area_sales_2' is not defined